In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import re
import yfinance
from pandas_datareader import data as pdr

yfinance.pdr_override()

# Get html code of website that has stock list for S&P 500
url = 'https://www.slickcharts.com/sp500'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, 'html')

# Find the table holding the list of stocks
table = soup.find('table')
temp = pd.read_html(table.prettify())
sp = pd.DataFrame(temp[0])

# Put the stock symbols into one list
stockList = []
for idx, row in sp.iterrows():
    stockList.append(row['Symbol'])

# Set up time objects
now = dt.datetime.now()
start = dt.datetime(now.year-2, now.month, now.day)

# Go through each stock in list and test the purchasing setup
for stock in stockList:
    df = pdr.get_data_yahoo(stock,start,now)

    # Create column of rolling highs and lows (High/Low over past 10 days)
    df['L10'] = df['Low'].rolling(window=10).min()
    df['H10'] = df['High'].rolling(window=10).max()

    # Calculate %K and %D (part of stochastic measure)
    df['%K'] = 100*((df['Close'] - df['L10']) / (df['H10'] - df['L10']))
    df['%D'] = df['%K'].rolling(window=4).mean()
print('done')

NameError: name 'yf' is not defined